In [31]:
import os
os.environ['GROQ_API_KEY'] = 'YOUR_GROQ_API_KEY'
os.environ['TAVILY_API_KEY'] = 'YOUR_TAVILY_API_KEY'

In [36]:
!pip install -q langchain-groq langchain-community langchain-core requests tavily-python

In [33]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
import requests

In [51]:
# Importing a Tool for Web Search
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults()

results = search_tool.invoke('Who is Rohit Sharma')

In [43]:
results

[{'title': 'Rohit Sharma | Life, Cricket, Career, Awards, & Facts | Britannica',
  'url': 'https://www.britannica.com/biography/Rohit-Sharma',
  'content': 'Rohit Sharma (born April 30, 1987, Bansod, India) is an Indian international cricketer and the current captain of the Indian cricket team in the One-Day International (ODI) format. Sharma was also the captain of India’s Twenty20 International team until he announced his retirement on June 29, 2024, after leading the Indian side to its T20I World Cup victory. He retired from Test cricket on May 7, 2025, ending a three-year stint as skipper. He captained the Mumbai Indians, a franchise team in [...] Sharma is the only player to have scored three double hundreds in ODI cricket; no other player has scored even two. He also holds the record for most international sixes across all formats, most sixes in World Cups, and the highest individual score in an ODI. He was a member of the Indian sides that won the 2007 T20I World Cup, the 2013 I

In [44]:
llm = ChatGroq(model='llama-3.3-70b-versatile')

In [45]:
llm.invoke('Hi How are you')

AIMessage(content="Hello. I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you. How about you? How's your day going so far? Is there anything I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 39, 'total_tokens': 99, 'completion_time': 0.218181818, 'prompt_time': 0.001990026, 'queue_time': 0.218696754, 'total_time': 0.220171844}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run--18902972-d0d2-423b-8736-77c4b12b420f-0', usage_metadata={'input_tokens': 39, 'output_tokens': 60, 'total_tokens': 99})

In [46]:
# ReAct -> Reasoning + Action
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [47]:
prompt = hub.pull('hwchase17/react') # Pulls standard ReAct Agent Prompt

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [48]:
# Creating ReAct Agent manually with pulled prompt
agent = create_react_agent(
    llm=llm,
    prompt=prompt,
    tools=[search_tool]
)

In [49]:
# Wrapping it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True
)

In [50]:
response = agent_executor.invoke({'input' : 'Who is Rohit Sharma'})
print(response)



> Entering new AgentExecutor chain...
To find out who Rohit Sharma is, I should first search for information about him on a reliable search engine.

Action: tavily_search_results_json
Action Input: "Rohit Sharma"[{'title': 'Rohit Sharma | Life, Cricket, Career, Awards, & Facts | Britannica', 'url': 'https://www.britannica.com/biography/Rohit-Sharma', 'content': 'Rohit Sharma (born April 30, 1987, Bansod, India) is an Indian international cricketer and the current captain of the Indian cricket team in the One-Day International (ODI) format. Sharma was also the captain of India’s Twenty20 International team until he announced his retirement on June 29, 2024, after leading the Indian side to its T20I World Cup victory. He retired from Test cricket on May 7, 2025, ending a three-year stint as skipper. He captained the Mumbai Indians, a franchise team in [...] Encyclopedia Britannica\nEncyclopedia Britannica\nRohit Sharma\nRohit Sharma batting\nCar with a pickle design in the Zagreb Red B

In [53]:
response['output']

"Rohit Sharma is a renowned Indian international cricketer who has made a significant impact in the sport, widely regarded as one of the greatest ODI opening batters of all time. He is the current captain of the Indian cricket team in the One-Day International (ODI) format and has led India to victory at the 2024 Men's T20 World Cup."